In the resposity Analyzing-calcium-imaging-in-MATLAB we isolated the $\frac{\Delta F}{F}$ traces of individual neurons from calcium imaging videos, and then identified the peaks and troughs of calcium transients in the $\frac{\Delta F}{F}$ trace. In this notebook we will use the identified calcium transients to train a bayesian decoder to predict a mouse's position along a linear track. To start we need to convert the data from a '.mat' file to something that's readable in Python. I have already converted the relevant data to an easier to use form using the function genHMMcode.m. I have made a copy of this data in the data folder and called it hmmtest_c27_2NRE.mat. To start we will read in the data and convert it to a form that's easier to use in Python

In [7]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.ndimage.filters
import os
import sys
import warnings
import numpy as np
import time
import random
import seaborn as sns
import scipy.stats as stats
import h5py


#Read data in from MATLAB. Needs to be saved with option 'v7.3' to be correctly read in
notebook_path = os.path.abspath("Decoding activity with a Bayesian decoder.ipynb")
f = h5py.File(os.path.join(notebook_path, "\data\exampledata_c27_2NRE.txt"), 'r')
trandat = f['hmmtestspikesmooth']
timeDat =  f['microscopeTime']
posDat = f['microscopeLocation']
pDat = f['placecells']
pcells = np.array(pDat[()])
fulldat = np.array(trandat[()])
mTime = np.array(timeDat[()])
mPos = np.array(posDat[()])
epochst = np.array(etDat[()])
epochsnt = np.array(ntDat[()])
allepochs = np.array(alldat[()])
f.close()
#Conver data to formats that nelpy can use


OSError: Unable to open file (unable to open file: name = 'C:\data\exampledata_c27_2NRE.txt', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)